# Landslide susceptibility analysis of roads in Kyrgyzstan


# Introduction

add text and short describtion

## Preparation, Data Download and Package Installing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install geopandas
!pip install rasterio
!pip install shapely
!pip install gdal
!pip install numpy
!pip install matplotlib
!pip install rtree
!pip install gdal
!pip install scipy
!pip install rioxarray
!pip install netCDF4
!pip install fiona
!pip install dask rasterio dask[distributed]
!pip install richdem
!pip install xdem
!pip install whitebox

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon, LineString, mapping, box
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
import matplotlib.pyplot as plt
import json
import requests
import pandas as pd
import base64
import getopt
import itertools
import math
import netrc
import os
import ssl
import sys
import time
from getpass import getpass
import xarray as xr
import rioxarray
import netCDF4
import h5netcdf
import fiona
from pyproj import CRS, Transformer
import zipfile
from whitebox import WhiteboxTools
import tempfile

##Downloads

can be skipped, find the downloads in 00_input

In [ ]:
#!/usr/bin/env python
# ----------------------------------------------------------------------------
# NSIDC Data Download Script
#
# Copyright (c) 2024 Regents of the University of Colorado
# Permission is hereby granted, free of charge, to any person obtaining
# a copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
# The above copyright notice and this permission notice shall be included
# in all copies or substantial portions of the Software.
#
# Tested in Python 2.7 and Python 3.4, 3.6, 3.7, 3.8, 3.9
#
# To run the script at a Linux, macOS, or Cygwin command-line terminal:
#   $ python nsidc-data-download.py
#
# On Windows, open Start menu -> Run and type cmd. Then type:
#     python nsidc-data-download.py
#
# The script will first search Earthdata for all matching files.
# You will then be prompted for your Earthdata username/password
# and the script will download the matching files.
#
# If you wish, you may store your Earthdata username/password in a .netrc
# file in your $HOME directory and the script will automatically attempt to
# read this file. The .netrc file should have the following format:
#    machine urs.earthdata.nasa.gov login MYUSERNAME password MYPASSWORD
# where 'MYUSERNAME' and 'MYPASSWORD' are your Earthdata credentials.
#
# Instead of a username/password, you may use an Earthdata bearer token.
# To construct a bearer token, log into Earthdata and choose "Generate Token".
# To use the token, when the script prompts for your username,
# just press Return (Enter). You will then be prompted for your token.
# You can store your bearer token in the .netrc file in the following format:
#    machine urs.earthdata.nasa.gov login token password MYBEARERTOKEN
# where 'MYBEARERTOKEN' is your Earthdata bearer token.
#
from __future__ import print_function

import base64
import getopt
import itertools
import json
import math
import netrc
import os.path
import ssl
import sys
import time
from getpass import getpass

try:
    from urllib.parse import urlparse
    from urllib.request import urlopen, Request, build_opener, HTTPCookieProcessor
    from urllib.error import HTTPError, URLError
except ImportError:
    from urlparse import urlparse
    from urllib2 import urlopen, Request, HTTPError, URLError, build_opener, HTTPCookieProcessor

short_name = 'HMA_DEM8m_MOS'
version = '1'
time_start = '2002-01-28T00:00:00Z'
time_end = '2016-11-24T23:59:59Z'
bounding_box = '68,39,81,44'
polygon = ''
filename_filter = ''
url_list = []

CMR_URL = 'https://cmr.earthdata.nasa.gov'
URS_URL = 'https://urs.earthdata.nasa.gov'
CMR_PAGE_SIZE = 2000
CMR_FILE_URL = ('{0}/search/granules.json?provider=NSIDC_ECS'
                '&sort_key[]=start_date&sort_key[]=producer_granule_id'
                '&scroll=true&page_size={1}'.format(CMR_URL, CMR_PAGE_SIZE))


def get_username():
    username = ''

    # For Python 2/3 compatibility:
    try:
        do_input = raw_input  # noqa
    except NameError:
        do_input = input

    username = do_input('Earthdata username (or press Return to use a bearer token): ')
    return username


def get_password():
    password = ''
    while not password:
        password = getpass('password: ')
    return password


def get_token():
    token = ''
    while not token:
        token = getpass('bearer token: ')
    return token


def get_login_credentials():
    """Get user credentials from .netrc or prompt for input."""
    credentials = None
    token = None

    try:
        info = netrc.netrc()
        username, account, password = info.authenticators(urlparse(URS_URL).hostname)
        if username == 'token':
            token = password
        else:
            credentials = '{0}:{1}'.format(username, password)
            credentials = base64.b64encode(credentials.encode('ascii')).decode('ascii')
    except Exception:
        username = None
        password = None

    if not username:
        username = get_username()
        if len(username):
            password = get_password()
            credentials = '{0}:{1}'.format(username, password)
            credentials = base64.b64encode(credentials.encode('ascii')).decode('ascii')
        else:
            token = get_token()

    return credentials, token


def build_version_query_params(version):
    desired_pad_length = 3
    if len(version) > desired_pad_length:
        print('Version string too long: "{0}"'.format(version))
        quit()

    version = str(int(version))  # Strip off any leading zeros
    query_params = ''

    while len(version) <= desired_pad_length:
        padded_version = version.zfill(desired_pad_length)
        query_params += '&version={0}'.format(padded_version)
        desired_pad_length -= 1
    return query_params


def filter_add_wildcards(filter):
    if not filter.startswith('*'):
        filter = '*' + filter
    if not filter.endswith('*'):
        filter = filter + '*'
    return filter


def build_filename_filter(filename_filter):
    filters = filename_filter.split(',')
    result = '&options[producer_granule_id][pattern]=true'
    for filter in filters:
        result += '&producer_granule_id[]=' + filter_add_wildcards(filter)
    return result


def build_cmr_query_url(short_name, version, time_start, time_end,
                        bounding_box=None, polygon=None,
                        filename_filter=None):
    params = '&short_name={0}'.format(short_name)
    params += build_version_query_params(version)
    params += '&temporal[]={0},{1}'.format(time_start, time_end)
    if polygon:
        params += '&polygon={0}'.format(polygon)
    elif bounding_box:
        params += '&bounding_box={0}'.format(bounding_box)
    if filename_filter:
        params += build_filename_filter(filename_filter)
    return CMR_FILE_URL + params


def get_speed(time_elapsed, chunk_size):
    if time_elapsed <= 0:
        return ''
    speed = chunk_size / time_elapsed
    if speed <= 0:
        speed = 1
    size_name = ('', 'k', 'M', 'G', 'T', 'P', 'E', 'Z', 'Y')
    i = int(math.floor(math.log(speed, 1000)))
    p = math.pow(1000, i)
    return '{0:.1f}{1}B/s'.format(speed / p, size_name[i])


def output_progress(count, total, status='', bar_len=60):
    if total <= 0:
        return
    fraction = min(max(count / float(total), 0), 1)
    filled_len = int(round(bar_len * fraction))
    percents = int(round(100.0 * fraction))
    bar = '=' * filled_len + ' ' * (bar_len - filled_len)
    fmt = '  [{0}] {1:3d}%  {2}   '.format(bar, percents, status)
    print('\b' * (len(fmt) + 4), end='')  # clears the line
    sys.stdout.write(fmt)
    sys.stdout.flush()


def cmr_read_in_chunks(file_object, chunk_size=1024 * 1024):
    """Read a file in chunks using a generator. Default chunk size: 1Mb."""
    while True:
        data = file_object.read(chunk_size)
        if not data:
            break
        yield data


def get_login_response(url, credentials, token):
    opener = build_opener(HTTPCookieProcessor())

    req = Request(url)
    if token:
        req.add_header('Authorization', 'Bearer {0}'.format(token))
    elif credentials:
        try:
            response = opener.open(req)
            # We have a redirect URL - try again with authorization.
            url = response.url
        except HTTPError:
            # No redirect - just try again with authorization.
            pass
        except Exception as e:
            print('Error{0}: {1}'.format(type(e), str(e)))
            sys.exit(1)

        req = Request(url)
        req.add_header('Authorization', 'Basic {0}'.format(credentials))

    try:
        response = opener.open(req)
    except HTTPError as e:
        err = 'HTTP error {0}, {1}'.format(e.code, e.reason)
        if 'Unauthorized' in e.reason:
            if token:
                err += ': Check your bearer token'
            else:
                err += ': Check your username and password'
        print(err)
        sys.exit(1)
    except Exception as e:
        print('Error{0}: {1}'.format(type(e), str(e)))
        sys.exit(1)

    return response


def cmr_download(urls, force=False, quiet=False):
    """Download files from list of urls."""
    if not urls:
        return

    url_count = len(urls)
    if not quiet:
        print('Downloading {0} files...'.format(url_count))
    credentials = None
    token = None
    save_dir = '/content/drive/MyDrive/disaster_proj/tiff_files'
    os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist

    for index, url in enumerate(urls, start=1):
        if not credentials and not token:
            p = urlparse(url)
            if p.scheme == 'https':
                credentials, token = get_login_credentials()

        filename = os.path.join(save_dir, url.split('/')[-1])
        if not quiet:
            print('{0}/{1}: {2}'.format(str(index).zfill(len(str(url_count))),
                                        url_count, filename))

        try:
            response = get_login_response(url, credentials, token)
            length = int(response.headers['content-length'])
            try:
                if not force and length == os.path.getsize(filename):
                    if not quiet:
                        print('  File exists, skipping')
                    continue
            except OSError:
                pass
            count = 0
            chunk_size = min(max(length, 1), 1024 * 1024)
            max_chunks = int(math.ceil(length / chunk_size))
            time_initial = time.time()
            with open(filename, 'wb') as out_file:
                for data in cmr_read_in_chunks(response, chunk_size=chunk_size):
                    out_file.write(data)
                    if not quiet:
                        count = count + 1
                        time_elapsed = time.time() - time_initial
                        download_speed = get_speed(time_elapsed, count * chunk_size)
                        output_progress(count, max_chunks, status=download_speed)
            if not quiet:
                print()
        except HTTPError as e:
            print('HTTP error {0}, {1}'.format(e.code, e.reason))
        except URLError as e:
            print('URL error: {0}'.format(e.reason))
        except IOError:
            raise




def cmr_filter_urls(search_results):
    """Select only the desired data files from CMR response."""
    if 'feed' not in search_results or 'entry' not in search_results['feed']:
        return []

    entries = [e['links']
               for e in search_results['feed']['entry']
               if 'links' in e]
    # Flatten "entries" to a simple list of links
    links = list(itertools.chain(*entries))

    urls = []
    unique_filenames = set()
    for link in links:
        if 'href' not in link:
            # Exclude links with nothing to download
            continue
        if 'inherited' in link and link['inherited'] is True:
            # Why are we excluding these links?
            continue
        if 'rel' in link and 'data#' not in link['rel']:
            # Exclude links which are not classified by CMR as "data" or "metadata"
            continue

        if 'title' in link and 'opendap' in link['title'].lower():
            # Exclude OPeNDAP links--they are responsible for many duplicates
            # This is a hack; when the metadata is updated to properly identify
            # non-datapool links, we should be able to do this in a non-hack way
            continue

        filename = link['href'].split('/')[-1]
        if filename in unique_filenames:
            # Exclude links with duplicate filenames (they would overwrite)
            continue
        unique_filenames.add(filename)

        urls.append(link['href'])

    return urls


def cmr_search(short_name, version, time_start, time_end,
               bounding_box='', polygon='', filename_filter='', quiet=False):
    """Perform a scrolling CMR query for files matching input criteria."""
    cmr_query_url = build_cmr_query_url(short_name=short_name, version=version,
                                        time_start=time_start, time_end=time_end,
                                        bounding_box=bounding_box,
                                        polygon=polygon, filename_filter=filename_filter)
    if not quiet:
        print('Querying for data:\n\t{0}\n'.format(cmr_query_url))

    cmr_scroll_id = None
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    urls = []
    hits = 0
    while True:
        req = Request(cmr_query_url)
        if cmr_scroll_id:
            req.add_header('cmr-scroll-id', cmr_scroll_id)
        try:
            response = urlopen(req, context=ctx)
        except Exception as e:
            print('Error: ' + str(e))
            sys.exit(1)
        if not cmr_scroll_id:
            # Python 2 and 3 have different case for the http headers
            headers = {k.lower(): v for k, v in dict(response.info()).items()}
            cmr_scroll_id = headers['cmr-scroll-id']
            hits = int(headers['cmr-hits'])
            if not quiet:
                if hits > 0:
                    print('Found {0} matches.'.format(hits))
                else:
                    print('Found no matches.')
        search_page = response.read()
        search_page = json.loads(search_page.decode('utf-8'))
        url_scroll_results = cmr_filter_urls(search_page)
        if not url_scroll_results:
            break
        if not quiet and hits > CMR_PAGE_SIZE:
            print('.', end='')
            sys.stdout.flush()
        urls += url_scroll_results

    if not quiet and hits > CMR_PAGE_SIZE:
        print()
    return urls


def main(argv=None):
    global short_name, version, time_start, time_end, bounding_box, \
        polygon, filename_filter, url_list

    if argv is None:
        argv = sys.argv[1:]

    force = False
    quiet = False
    usage = 'usage: nsidc-download_***.py [--help, -h] [--force, -f] [--quiet, -q]'

    try:
        opts, args = getopt.getopt(argv, 'hfq', ['help', 'force', 'quiet'])
        for opt, _arg in opts:
            if opt in ('-f', '--force'):
                force = True
            elif opt in ('-q', '--quiet'):
                quiet = True
            elif opt in ('-h', '--help'):
                print(usage)
                sys.exit(0)
    except getopt.GetoptError as e:
        print(e.args[0])
        print(usage)
        sys.exit(1)

    # Supply some default search parameters, just for testing purposes.
    # These are only used if the parameters aren't filled in up above.
    if 'short_name' in short_name:
        short_name = 'ATL06'
        version = '003'
        time_start = '2018-10-14T00:00:00Z'
        time_end = '2021-01-08T21:48:13Z'
        bounding_box = ''
        polygon = ''
        filename_filter = '*ATL06_2020111121*'
        url_list = []

    try:
        if not url_list:
            url_list = cmr_search(short_name, version, time_start, time_end,
                                  bounding_box=bounding_box, polygon=polygon,
                                  filename_filter=filename_filter, quiet=quiet)

        cmr_download(url_list, force=force, quiet=quiet)
    except KeyboardInterrupt:
        quit()


if __name__ == '__main__':
    main()

In [ ]:
output_file = "/content/drive/MyDrive/disaster_proj/00_input/kg_admin/kyrgyzstan.geojson"
os.makedirs(os.path.dirname(output_file), exist_ok=True)
URL = "https://api.ohsome.org/v1/elements/geometry"
filter_query = "id:relation/178009"
timestamps = ["2023-06-01"]

for timestamp in timestamps:
    data = {"bboxes": None, "time": timestamp, "filter": filter_query}  # bboxes is not required for specific ID queries
    response = requests.post(URL, data=data)

    try:
        response_json = response.json()
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        continue

    if 'features' in response_json:
        features = response_json['features']
        gdf = gpd.GeoDataFrame.from_features(features)

        if gdf.empty:
            print("GeoDataFrame is empty.")
            continue

        gdf.set_crs("EPSG:4326", inplace=True)
        try:
            gdf.to_file(output_file, driver="GeoJSON")
            print(f"GeoDataFrame saved to output file {output_file}.")
        except Exception as e:
            print(f"An error occurred while saving the GeoDataFrame: {e}")
    else:
        print("No features found in the response.")


In [ ]:
output_file = "/content/drive/MyDrive/disaster_proj/00_input/kg_admin/kg_roads.geojson"
os.makedirs(os.path.dirname(output_file), exist_ok=True)

URL = "https://api.ohsome.org/v1/elements/geometry"
bbox = "68.1557281, 39.0246074, 80.5633959, 43.5459749"
filters = ["highway=* and type:way"]
timestamps = ["2023-06-01"]

for filter in filters:
    for timestamp in timestamps:
        data = {"bboxes": bbox, "time": timestamp, "filter": filter}
        response = requests.post(URL, data=data)

        try:
            response_json = response.json()
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            continue

        if 'features' in response_json:
            features = response_json['features']
            gdf = gpd.GeoDataFrame.from_features(features)

            if gdf.empty:
                print("GeoDataFrame is empty.")
                continue

            gdf.set_crs("EPSG:4326", inplace=True)

            gdf = gdf[gdf.geometry.type.isin(["LineString", "MultiLineString"])]
            if gdf.empty:
                print("No LineString or MultiLineString geometries found.")
                continue
            try:
                gdf.to_file(output_file, driver="GeoJSON")
                print(f"GeoDataFrame saved to output file {output_file}.")
            except Exception as e:
                print(f"An error occurred while saving the GeoDataFrame: {e}")
        else:
            print("No features found in the response.")

## Prepare Data

###Reprojecting to 32642

In [ ]:
def reproject_vector_to_epsg(input_file, output_file, epsg):
  gdf = gpd.read_file(input_file)
  gdf_reprojected = gdf.to_crs(epsg=epsg)
  gdf_reprojected.to_file(output_file, driver='GeoJSON')
  print(f'Reprojected vector file saved to {output_file}')

In [ ]:
def reproject_raster(input_path, output_path, target_crs):
    os.makedirs(output_path, exist_ok=True)
    if os.path.isfile(input_path):
        try:
            with rasterio.open(input_path) as src:
                transform, width, height = calculate_default_transform(
                    src.crs, target_crs, src.width, src.height, *src.bounds)
                meta = src.meta.copy()
                meta.update({
                    'crs': target_crs,
                    'transform': transform,
                    'width': width,
                    'height': height
                })
                name, ext = os.path.splitext(os.path.basename(input_path))
                output_filename = os.path.join(output_path, f"reprojected_{name}{ext}")
                with rasterio.open(output_filename, 'w', **meta) as dst:
                    for i in range(1, src.count + 1):
                        reproject(
                            source=rasterio.band(src, i),
                            destination=rasterio.band(dst, i),
                            src_transform=src.transform,
                            src_crs=src.crs,
                            dst_transform=transform,
                            dst_crs=target_crs,
                            resampling=Resampling.nearest)
        except Exception as e:
            print(f"Error reprojecting {input_path}: {e}")
    elif os.path.isdir(input_path):
        search_pattern = os.path.join(input_path, '*.tif')
        files = glob.glob(search_pattern)
        for file in files:
            try:
                with rasterio.open(file) as src:
                    transform, width, height = calculate_default_transform(
                        src.crs, target_crs, src.width, src.height, *src.bounds)
                    meta = src.meta.copy()
                    meta.update({
                        'crs': target_crs,
                        'transform': transform,
                        'width': width,
                        'height': height
                    })
                    name, ext = os.path.splitext(os.path.basename(file))
                    output_filename = os.path.join(output_path, f"resampled_{name}{ext}")
                    with rasterio.open(output_filename, 'w', **meta) as dst:
                        for i in range(1, src.count + 1):
                            reproject(
                                source=rasterio.band(src, i),
                                destination=rasterio.band(dst, i),
                                src_transform=src.transform,
                                src_crs=src.crs,
                                dst_transform=transform,
                                dst_crs=target_crs,
                                resampling=Resampling.nearest)
            except Exception as e:
                print(f"Error reprojecting {file}: {e}")
    else:
        print("Invalid input path.")

Rain data needs special handling

In [ ]:
def reproject_rain(input_path, output_path, target_crs):
    nc_file = xr.open_dataset(input_path)
    src_crs = '+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs +R=6371000'
    nc_file.rio.write_crs(src_crs, inplace=True)
    reprojected_data = nc_file['climatology-pr-annual-mean'].rio.reproject(target_crs, resampling=Resampling.bilinear)
    reprojected_data.rio.to_raster(output_path)
    print("Reprojection complete. Reprojected data saved to 'reprojected_rain.tif'")

### Clip to AOI
later also used for clipping to road buffer


In [ ]:
def clip_geom_to_kg(input_geom, clip_to_geom, output_file):
    input_geom = gpd.read_file(input_geom)
    clip_area = gpd.read_file(clip_to_geom)
    if input_geom.crs != clip_area.crs:
        clip_area = clip_area.to_crs(input_geom.crs)
    clipped_geom = gpd.clip(input_geom, clip_area)
    output_dir = os.path.dirname(output_file)
    os.makedirs(output_dir, exist_ok=True)
    clipped_geom.to_file(output_file, driver='GeoJSON')
    print(f'Clipped {input_geom} saved to {output_file}')

In [ ]:
def clip_raster_to_geom(input_path, boundary_path, output_path, value_range=(0, 10000)):
    boundary = gpd.read_file(boundary_path)
    shapes = [mapping(geometry) for geometry in boundary.geometry]
    with rasterio.open(input_path) as src:
        original_raster = src.read(1)
        out_image, out_transform = mask(src, shapes, crop=True, all_touched=True)
        out_meta = src.meta.copy()
        nodata_value = src.nodata if src.nodata is not None else np.nan
        out_image = np.where(out_image == nodata_value, np.nan, out_image)
        min_val, max_val = value_range
        out_image = np.clip(out_image, min_val, max_val)
        if len(out_image.shape) > 2:
            out_image = out_image[0]
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[0],
            "width": out_image.shape[1],
            "transform": out_transform,
            "nodata": nodata_value
        })
        with rasterio.Env(CHECK_DISK_FREE_SPACE=False): #big data
          with rasterio.open(output_path, "w", **out_meta) as dest:
              dest.write(out_image, 1)
    print(f"Clipped raster ({input_path}) saved to {output_path}")

In [ ]:
def clip_rasterfolder_to_geom(input_folder, boundary_path, output_folder, value_range=(0, 10000), default_nodata=-9999):
    boundary = gpd.read_file(boundary_path)
    shapes = [mapping(geometry) for geometry in boundary.geometry]
    os.makedirs(output_folder, exist_ok=True)

    for input_path in glob.glob(os.path.join(input_folder, "*.tif")):
        base_name = os.path.basename(input_path)
        output_path = os.path.join(output_folder, f"clipped_{base_name}")

        with rasterio.open(input_path) as src:
            raster_bounds = box(*src.bounds)
            raster_crs = src.crs

            if boundary.crs != raster_crs:
                boundary = boundary.to_crs(raster_crs)

            overlap = boundary.geometry.intersects(raster_bounds)
            if not any(overlap):
                print(f"No overlap between raster {base_name} and boundary. Skipping.")
                continue

            out_image, out_transform = mask(src, shapes, crop=True, all_touched=True)
            out_meta = src.meta.copy()
            nodata_value = src.nodata if src.nodata is not None else default_nodata

            out_image = np.where(out_image == nodata_value, default_nodata, out_image)

            min_val, max_val = value_range
            out_image = np.clip(out_image, min_val, max_val)

            if len(out_image.shape) > 2:
                out_image = out_image[0]

            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[0],
                "width": out_image.shape[1],
                "transform": out_transform,
                "nodata": default_nodata,
                "dtype": "float32"
            })

            with rasterio.Env(CHECK_DISK_FREE_SPACE=False):  # For large data
                with rasterio.open(output_path, "w", **out_meta) as dest:
                    dest.write(out_image, 1)

        print(f"Clipped raster {input_path} saved to {output_path}")

    print("All rasters clipped and saved.")

Permaforst needs special handling and additional masking

In [ ]:
def clip_pf_to_geom(input_path, boundary_path, output_path, value_range=(0, 10000)):
    boundary = gpd.read_file(boundary_path)
    shapes = [mapping(geometry) for geometry in boundary.geometry]
    with rasterio.open(input_path) as src:
        out_image, out_transform = mask(src, shapes, crop=True, all_touched=True)
        out_meta = src.meta.copy()
        nodata_value = src.nodata if src.nodata is not None else -9999
        out_image = np.where(out_image == nodata_value, np.nan, out_image)
        min_val, max_val = value_range
        out_image = np.clip(out_image, min_val, max_val)
        if out_image.ndim > 2:
            out_image = out_image[0]
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[0],
            "width": out_image.shape[1],
            "transform": out_transform,
            "nodata": nodata_value
        })
        with rasterio.Env(CHECK_DISK_FREE_SPACE=False):
            with rasterio.open(output_path, "w", **out_meta) as dest:
                dest.write(out_image, 1)
    print(f"Clipped permafrost saved to {output_path}")


In [ ]:
def mask_pf(input_path, output_path):
    with rasterio.open(input_path) as src:
        data = src.read(1)
        meta = src.meta.copy()
        data[data == 0] = -9999
        meta.update({'nodata': -9999})
        with rasterio.open(output_path, 'w', **meta) as dst:
            dst.write(data, 1)
    print(f"Masked raster saved to {output_path}")

###Simplify and buffer roads

In [ ]:
import os
import geopandas as gpd
from shapely.ops import unary_union
import matplotlib.pyplot as plt

def buffer_roads(input_file, output_file):
    os.environ['OGR_GEOJSON_MAX_OBJ_SIZE'] = '0'
    roads = gpd.read_file(input_file)
    print("File read successfully!")
    roads_simplified = roads.simplify(tolerance=500, preserve_topology=True)
    roads_buffered = roads_simplified.buffer(3000)
    merged_buffered_roads = unary_union(roads_buffered)
    merged_gdf = gpd.GeoDataFrame(geometry=[merged_buffered_roads], crs=32642)
    merged_gdf.to_file(output_file, driver='GeoJSON')
    print("Merged buffered roads saved successfully!")

##DEM derivates

###calculate curvature from clipped DEM

In [ ]:
def calc_curvature(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    wbt = WhiteboxTools()
    wbt.work_dir = input_folder

    for dem_file in glob.glob(os.path.join(input_folder, "*.tif")):
        output_file = os.path.join(output_folder, os.path.basename(dem_file).replace(".tif", "_curvature.tif"))

        wbt.plan_curvature(dem_file, output_file, log=False, zfactor=None)
        print(f"Processed and saved: {dem_file} -> {output_file}")

    print(f"All files processed and saved in folder: {output_folder}")

###calculate slope from clipped DEM

In [ ]:
def calc_slope(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    wbt = WhiteboxTools()
    wbt.work_dir = input_folder

    for dem_file in glob.glob(os.path.join(input_folder, "*.tif")):
        output_file = os.path.join(output_folder, os.path.basename(dem_file).replace(".tif", "_slope.tif"))

        wbt.slope(dem_file, output_file, units='DEGREES')  # You can change 'DEGREES' to 'PERCENT' if needed
        print(f"Processed and saved: {dem_file} -> {output_file}")

    print(f"All files processed and saved in folder: {output_folder}")

## Merging

In [ ]:
def merge_rasters_gdal(input_dir, output_file):
  os.makedirs(output_dir, exist_ok=True)
    rasters = glob.glob(os.path.join(input_dir, "*.tif"))

    if not rasters:
        print(f"No raster files found in {input_dir}. Skipping.")
        return

    vrt_options = gdal.BuildVRTOptions(resampleAlg='bilinear', addAlpha=False)
    vrt = gdal.BuildVRT('', rasters, options=vrt_options)

    gdal.Translate(output_file, vrt, bandList=[1])

    print(f"Merged {len(rasters)} DEM rasters into {output_file}")

print("Merging completed!")

In [ ]:
def merge_rasters_lulc(input_dir, category, output_path):
    raster_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if category in f and f.endswith('.tif')]
    if not raster_files:
        print(f"No files found for category: {category}")
        return

    gdal_command = ['gdal_merge.py', '-o', output_path, '-n', '0', '-a_nodata', '0'] + raster_files
    os.system(' '.join(gdal_command))
    print(f"Merged {category} files saved to {output_path}")

print("Merging completed!")


##Resample rasters

In [ ]:
def resample_raster(input_path, output_path, target_resolution):
  with rasterio.Env(CHECK_DISK_FREE_SPACE=False):
    with rasterio.open(input_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, src.crs, src.width, src.height, *src.bounds, resolution=target_resolution
        )
        meta = src.meta.copy()
        meta.update({
            'crs': src.crs,
            'transform': transform,
            'width': width,
            'height': height,
            'nodata': -9999
        })
        with rasterio.open(output_path, 'w', **meta) as dst:
            for band in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, band),
                    destination=rasterio.band(dst, band),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=src.crs,
                    resampling=Resampling.nearest
                )

In [ ]:
def resample_raster_pf(input_path, output_path, target_resolution):
    with rasterio.Env(CHECK_DISK_FREE_SPACE=False):
        with rasterio.open(input_path) as src:
            transform, width, height = calculate_default_transform(
                src.crs, src.crs, src.width, src.height, *src.bounds, resolution=target_resolution)
            meta = src.meta.copy()
            meta.update({
                'crs': src.crs,
                'transform': transform,
                'width': width,
                'height': height,
                'dtype': 'int16',
                'nodata': -9999
            })

            with rasterio.open(output_path, 'w', **meta) as dst:
                for band in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, band),
                        destination=rasterio.band(dst, band),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=src.crs,
                        resampling=Resampling.nearest
                    )

    print(f"Resampled raster saved to {output_path}")

In [ ]:
def resample_rasters_folder(input_folder, output_folder, target_resolution=30):
    os.makedirs(output_folder, exist_ok=True)

    for input_path in glob.glob(os.path.join(input_folder, "*.tif")):
        base_name = os.path.basename(input_path)
        output_path = os.path.join(output_folder, f"resampled_{base_name}")

        with rasterio.Env(CHECK_DISK_FREE_SPACE=False):
            with rasterio.open(input_path) as src:
                transform, width, height = calculate_default_transform(
                    src.crs, src.crs, src.width, src.height, *src.bounds, resolution=target_resolution
                )
                meta = src.meta.copy()
                dtype = meta["dtype"]
                if dtype in ["uint8", "uint16"]:
                    nodata_value = 0
                elif dtype in ["int16", "int32"]:
                    nodata_value = -9999
                else:
                    nodata_value = -9999.0

                meta.update({
                    'transform': transform,
                    'width': width,
                    'height': height,
                    'nodata': nodata_value
                })

                with rasterio.open(output_path, 'w', **meta) as dst:
                    for band in range(1, src.count + 1):
                        reproject(
                            source=rasterio.band(src, band),
                            destination=rasterio.band(dst, band),
                            src_transform=src.transform,
                            src_crs=src.crs,
                            dst_transform=transform,
                            dst_crs=src.crs,
                            resampling=Resampling.nearest
                        )
                print(f"Resampled and saved: {output_path}")
    print("All files resampled and saved.")

##Classification

Classification is necessary, due to the common permafrost classification scheme. Continuous, discontinuous, sporadic and isolated permafrost.

In [ ]:
def reclassify_raster(input_file, output_file):
    with rasterio.open(input_file) as src:
        data = src.read(1)
        no_data = src.nodata
        if no_data is not None:
            mask_nodata = data == no_data
        else:
            mask_nodata = np.zeros_like(data, dtype=bool)

        reclassified_data = np.copy(data).astype(float)

        reclassified_data[data <= 50] = 1.5  # isolated and sporadic
        reclassified_data[(data > 50) & (data <= 90)] = 2.5  # discontinous
        reclassified_data[data > 90] = 1.2  # continous

        reclassified_data[mask_nodata] = no_data

        profile = src.profile
        profile.update(dtype=rasterio.float32)

        with rasterio.Env(CHECK_DISK_FREE_SPACE=False):
          with rasterio.open(output_file, 'w', **profile) as dst:
              dst.write(reclassified_data.astype(rasterio.float32), 1)
              print(f"Reclassified raster saved to {output_file}")

##Normalization and Interpolization

In [ ]:
def normalize_lulc_file(input_file, output_folder):
    raw_weights = {
        'sparse_vegetation': -210.4,
        'dense_forest': -284.1,
        'snow': 6.53,
        'water': -67.0,
        'settlement': -20.5,
        'barren_land': 185.1,
        'fallow_land': 174.3,
        'agriculture': 79.0
    }

    lulc_codes = {
        1: 'water',              # Water
        2: 'dense_forest',       # Dense forest
        4: None,                 # Flooded vegetation (no class)
        5: 'agriculture',        # Agriculture
        7: 'settlement',         # Settlement
        8: 'barren_land',        # Barren land
        9: 'snow',               # Snow
        10: None,                # Clouds (no data)
        11: 'sparse_vegetation'  # Sparse vegetation
    }

    beta = 0.005

    def normalize_weight(raw_weight, beta):
        return math.exp(beta * raw_weight)

    normalized_weights = {key: normalize_weight(value, beta) for key, value in raw_weights.items()}

    os.makedirs(output_folder, exist_ok=True)

    output_path = os.path.join(output_folder, f'normalized_{os.path.basename(input_file)}')

    with rasterio.open(input_file) as src:
        lulc_data = src.read(1)
        normalized_susceptibility = np.zeros(lulc_data.shape, dtype=np.float32)

        for lulc_value, lulc_class in lulc_codes.items():
            if lulc_class is not None:
                normalized_susceptibility[lulc_data == lulc_value] = normalized_weights[lulc_class]

        out_meta = src.meta.copy()
        out_meta.update(dtype=rasterio.float32, count=1)

    with rasterio.open(output_path, 'w', **out_meta) as dst:
        dst.write(normalized_susceptibility, 1)

    print(f"Normalized LULC raster saved to {output_path}")

In [ ]:
def extract_zip_if_needed(input_path, extraction_dir):
    if input_path.endswith(".zip"):
        with zipfile.ZipFile(input_path, "r") as zip_ref:
            zip_ref.extractall(extraction_dir)
            print(f"Extracted {input_path} to {extraction_dir}")
        return extraction_dir
    return input_path


def interpolate_datapoints(data_type):
    df = pd.read_csv('/content/drive/MyDrive/disaster_proj/00_input/data_points_for_landslide_density.csv')
    x = df[df['type'] == data_type]['x'].values
    y = df[df['type'] == data_type]['y'].values
    pchip = PchipInterpolator(x, y)
    return pchip


def density_values(pchip, raster_input, output_file):
    with rasterio.open(raster_input) as src:
        raster_data = src.read(1)
        raster_data = ma.masked_where(raster_data == src.nodata, raster_data)
        landslide_density = pchip(raster_data)
        landslide_density = ma.masked_where(raster_data.mask, landslide_density)

        with rasterio.open(output_file, 'w', **src.meta) as dst:
            dst.write(landslide_density, 1)
            print(f"Landslide density raster saved to {output_file}")

    return landslide_density


def interpolate(input_paths, output_folder, data_types):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for idx, input_path in enumerate(input_paths):
        print(f"Processing: {input_path}")

        temp_folder = f"{output_folder}/temp_extracted_{idx}"
        processed_folder = extract_zip_if_needed(input_path, temp_folder)

        try:
            if input_path.endswith('clipped_to_road_buffer_rain.tif'):
                output_file = os.path.join(output_folder, f"precipitation_density.tif")
                pchip = interpolate_datapoints('precipitation')
                density_values(pchip, processed_folder, output_file)
                continue

            raster_files = [
                os.path.join(processed_folder, file)
                for file in os.listdir(processed_folder) if file.endswith('.tif')
            ] if os.path.isdir(processed_folder) else [processed_folder]

            if not raster_files:
                print(f"No raster files found in {processed_folder}. Skipping.")
                continue

            data_type = data_types[idx]

            pchip = interpolate_datapoints(data_type)

            for raster_file in raster_files:
                base_name = os.path.basename(raster_file)
                tile_number = base_name.split('_')[1].replace('tile-', '')

                output_file = os.path.join(output_folder, f"{data_type}_{base_name}")

                density_values(pchip, raster_file, output_file)

        finally:
            if os.path.exists(temp_folder):
                shutil.rmtree(temp_folder)
                print(f"Temporary folder {temp_folder} deleted.")


## Landslide susceptibility
Weight factors and calculate landslide susceptibility.

In [ ]:
import os
import zipfile
import shutil
import rasterio
import numpy as np
import glob

def weighting(lulc_file, permafrost_file, density_folder, output_zip_file):
    weights = {
        "slope": 0.3,
        "permafrost": 0.1,
        "precipitation": 0.1,
        "elevation": 0.3,
        "curvature": 0.1,
        "lulc": 0.1
    }

    output_temp_folder = "/content/drive/MyDrive/disaster_proj/temp_outputs"
    os.makedirs(output_temp_folder, exist_ok=True)

    def process_raster_in_chunks(input_file, weight, output_file):
      print(f"Processing file: {input_file} with weight: {weight}")
      with rasterio.open(input_file) as src:
          meta = src.meta
          meta.update(dtype=rasterio.float32)

          nodata_value = src.nodata
          data = src.read(1).astype(np.float32)

          custom_nodata_values = [-9999, -999, -9999.0]
          for val in custom_nodata_values:
              data[data == val] = np.nan

          if nodata_value is not None:
              data[data == nodata_value] = np.nan

          weighted_data = data * weight

          print(f"Value range of {input_file} after applying weight: min={np.nanmin(weighted_data)}, max={np.nanmax(weighted_data)}")

          with rasterio.open(output_file, 'w', **meta) as dst:
              for ji, window in src.block_windows(1):
                  block_data = src.read(1, window=window).astype(np.float32)

                  for val in custom_nodata_values:
                      block_data[block_data == val] = np.nan
                  if nodata_value is not None:
                      block_data[block_data == nodata_value] = np.nan

                  weighted_block_data = block_data * weight
                  dst.write(weighted_block_data.astype(rasterio.float32), 1, window=window)

      print(f"Saved weighted raster to: {output_file}")

    temp_files = []

    print("Processing LULC file...")
    output_file = os.path.join(output_temp_folder, f"weighted_{os.path.basename(lulc_file)}")
    process_raster_in_chunks(lulc_file, weights["lulc"], output_file)
    temp_files.append(output_file)

    print("Processing Permafrost file...")
    output_file = os.path.join(output_temp_folder, f"weighted_{os.path.basename(permafrost_file)}")
    process_raster_in_chunks(permafrost_file, weights["permafrost"], output_file)
    temp_files.append(output_file)

    print("Processing Density files...")
    for key in ["slope", "curvature", "precipitation", "elevation"]:
        density_files = glob.glob(os.path.join(density_folder, f"*{key}*.tif"))
        print(f"Found {len(density_files)} files for {key}.")
        for density_file in density_files:
            output_file = os.path.join(output_temp_folder, f"weighted_{os.path.basename(density_file)}")
            process_raster_in_chunks(density_file, weights[key], output_file)
            temp_files.append(output_file)

    print("Zipping output files...")
    with zipfile.ZipFile(output_zip_file, 'w') as zf:
        for temp_file in temp_files:
            zf.write(temp_file, arcname=os.path.basename(temp_file))
            os.remove(temp_file)
            print(f"Deleted temporary file: {temp_file}")

    print(f"All files have been processed and saved in: {output_zip_file}")

In [ ]:
import zipfile
import os

def unzip_file(input_zip_file, extract_to_folder):
    os.makedirs(extract_to_folder, exist_ok=True)

    with zipfile.ZipFile(input_zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)

    print(f"Files extracted to: {extract_to_folder}")

fill nodata of permafrost with zero

In [ ]:
import rasterio
import numpy as np

def fill_permafrost_with_ref_values(permafrost_file, reference_file, output_file):
    with rasterio.open(permafrost_file) as permafrost_src, rasterio.open(reference_file) as reference_src:
        permafrost_data = permafrost_src.read(1)
        reference_data = reference_src.read(1)

        mask_nan_permafrost = np.isnan(permafrost_data)
        mask_valid_reference = ~np.isnan(reference_data)

        permafrost_data[mask_nan_permafrost & mask_valid_reference] = 0

        meta = permafrost_src.meta.copy()
        meta.update(dtype=rasterio.float32, nodata=-999.0)

    with rasterio.open(output_file, 'w', **meta) as output_src:
        output_src.write(permafrost_data, 1)

    print(f"Filled permafrost raster saved to: {output_file}")

In [ ]:
import rasterio
from rasterio.warp import reproject, Resampling
import numpy as np

def align_and_sum_rasters(raster_files, output_file):
    with rasterio.open(raster_files[0]) as ref_src:
        ref_meta = ref_src.meta.copy()
        ref_meta.update(dtype='float32')
        ref_transform = ref_src.transform
        ref_width = ref_src.width
        ref_height = ref_src.height
        ref_crs = ref_src.crs

    sum_array = np.zeros((ref_height, ref_width), dtype=np.float32)

    for raster_path in raster_files:
        print(f"Processing raster: {raster_path}")
        with rasterio.open(raster_path) as src:
            aligned_data = np.zeros((ref_height, ref_width), dtype=np.float32)

            reproject(
                source=rasterio.band(src, 1),
                destination=aligned_data,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=ref_transform,
                dst_crs=ref_crs,
                resampling=Resampling.nearest,
            )

            sum_array += aligned_data

    with rasterio.open(output_file, 'w', **ref_meta) as dst:
        dst.write(sum_array, 1)

    print(f"Output saved to {output_file}")


## Call functions in correct order

# Execute

##1. Reproject

###reproject vector

In [ ]:
# admin boundary
reproject_vector_to_epsg('/content/drive/MyDrive/disaster_proj/00_input/kg_admin/kyrgyzstan.geojson', '/content/drive/MyDrive/disaster_proj/01_reprojected/reprojected_kyrgyzstan.geojson', 32642)
# roads
reproject_vector_to_epsg('/content/drive/MyDrive/disaster_proj/00_input/kg_admin/kg_roads.geojson', '/content/drive/MyDrive/disaster_proj/01_reprojected/reprojected_kg_roads.geojson', 32642)

###reproject raster

In [ ]:
# permafrost
reproject_raster('/content/drive/MyDrive/disaster_proj/00_input/permafrost.tif', '/content/drive/MyDrive/disaster_proj/01_reprojected/reprojected_permafrost', 32642)
# reproject rain
reproject_rain('/content/drive/MyDrive/disaster_proj/00_input/climatology-pr-annual-mean_cmip6-x0.25_hadgem3-gc31-mm-r1i1p1f3-historical_climatology_mean_1995-2014.nc', '/content/drive/MyDrive/disaster_proj/01_reprojected/reprojected_rain.tif', 32642)

##2. Clip to Kyrgyzstan (AOI)

In [ ]:
# roads
clip_geom_to_kg('/content/drive/MyDrive/disaster_proj/01_reprojected/reprojected_kg_roads.geojson', '/content/drive/MyDrive/disaster_proj/01_reprojected/reprojected_kyrgyzstan.geojson', '/content/drive/MyDrive/disaster_proj/02_clipped_to_KG/clipped_roads.geojson')
# rain
clip_raster_to_geom('/content/drive/MyDrive/disaster_proj/01_reprojected/reprojected_rain.tif', '/content/drive/MyDrive/disaster_proj/01_reprojected/reprojected_kyrgyzstan.geojson', '/content/drive/MyDrive/disaster_proj/02_clipped_to_KG/clipped_rain.tif')

##3. Simplify and buffer roads

In [ ]:
# buffer roads
buffer_roads('/content/drive/MyDrive/disaster_proj/02_clipped_to_KG/clipped_roads.geojson', '/content/drive/MyDrive/disaster_proj/04_buffered_roads/merged_buffered_roads_3000.geojson')

##4. DEM derrivates

###Slope

In [ ]:
calc_slope(input_folder="/content/drive/MyDrive/disaster_proj/00_input/dem_files", output_folder="/content/drive/MyDrive/disaster_proj/05_DEM_derivates/slope")

###Curvature

In [ ]:
calc_curvature(input_folder="/content/drive/MyDrive/disaster_proj/00_input/dem_files", output_folder="/content/drive/MyDrive/disaster_proj/05_DEM_derivates/curvature")

## 5. Merge Raster files

In [ ]:
input_dir = '/content/drive/MyDrive/disaster_proj/05_DEM_derivates/curvature'
output_dir = '/content/drive/MyDrive/disaster_proj/merged_rasters'
output = os.path.join(output_dir, "curvature_merged.tif")
merge_rasters_gdal(input_dir, output)

In [ ]:
input_dir = '/content/drive/MyDrive/disaster_proj/05_DEM_derivates/slope'
output_dir = '/content/drive/MyDrive/disaster_proj/merged_rasters'
output = os.path.join(output_dir, "slope_merged.tif")
merge_rasters_gdal(input_dir, output)

In [ ]:
input_dir = '/content/drive/MyDrive/disaster_proj/00_input/dem_files'
output_dir = '/content/drive/MyDrive/disaster_proj/merged_rasters'
output = os.path.join(output_dir, "elevation_merged.tif")
merge_rasters_gdal(input_dir, output)

In [ ]:
density_dir = '/content/drive/MyDrive/disaster_proj/08_reclass_norm/density'
lulc_dir = '/content/drive/MyDrive/disaster_proj/00_input/LULC_rasters'
output_dir = '/content/drive/MyDrive/disaster_proj/merged_rasters'
os.makedirs(output_dir, exist_ok=True)
lulc_output = os.path.join(output_dir, "lulc_merged.tif")
merge_rasters_lulc(lulc_dir, "", lulc_output)

## 6. Reproject Raster files

Curvature

In [ ]:
reproject_raster('/content/drive/MyDrive/disaster_proj/merged_rasters/curvature_merged.tif', '/content/drive/MyDrive/disaster_proj/01_reprojected/reprojected_curvature', 32642)

Slope

In [ ]:
reproject_raster('/content/drive/MyDrive/disaster_proj/merged_rasters/slope_merged.tif', '/content/drive/MyDrive/disaster_proj/01_reprojected/reprojected_slope', 32642)

Elevation

In [ ]:
reproject_raster('/content/drive/MyDrive/disaster_proj/merged_rasters/elevation_merged.tif', '/content/drive/MyDrive/disaster_proj/01_reprojected/reprojected_elevation', 32642)

LULC

In [ ]:
reproject_raster('/content/drive/MyDrive/disaster_proj/merged_rasters/lulc_merged.tif', '/content/drive/MyDrive/disaster_proj/01_reprojected/reprojected_lulc', 32642)

Precipitation

In [ ]:
reproject_raster('/content/drive/MyDrive/disaster_proj/02_clipped_to_KG/clipped_rain.tif', '/content/drive/MyDrive/disaster_proj/01_reprojected/reprojected_rain', 32642)

##7. Resampling

DEM

In [ ]:
resample_raster('/content/drive/MyDrive/disaster_proj/01_reprojected/reprojected_elevation', '/content/drive/MyDrive/disaster_proj/07_resampled/resampled_elevation.tif', 30)

Slope

In [ ]:
resample_raster('/content/drive/MyDrive/disaster_proj/01_reprojected/reprojected_slope', '/content/drive/MyDrive/disaster_proj/07_resampled/resampled_slope.tif', 30)

Curvature

In [ ]:
resample_raster('/content/drive/MyDrive/disaster_proj/01_reprojected/reprojected_curvature', '/content/drive/MyDrive/disaster_proj/07_resampled/resampled_curvature.tif', 30)

LULC

In [ ]:
resample_raster('/content/drive/MyDrive/disaster_proj/01_reprojected/reprojected_lulc', '/content/drive/MyDrive/disaster_proj/07_resampled/resampled_lulc.tif', 30)

Precipitation

In [ ]:
resample_raster('/content/drive/MyDrive/disaster_proj/01_reprojected/reprojected_rain', '/content/drive/MyDrive/disaster_proj/07_resampled/resampled_rain.tif', 30)

Permafrost

In [ ]:
resample_raster_pf('/content/drive/MyDrive/disaster_proj/01_reprojected/reprojected_permafrost.tif', '/content/drive/MyDrive/disaster_proj/07_resampled/resampled_permafrost.tif', 30)

##8. Clip to road buffer

###Slope and Curvature

Slope

In [ ]:
extract_and_process_zip('/content/drive/MyDrive/disaster_proj/05_DEM_derivates/slope_outputs.zip', '/content/drive/MyDrive/disaster_proj/04_buffered_roads/merged_buffered_roads_3000.geojson', '/content/drive/MyDrive/disaster_proj/06_clipped_to_road_buffer/clipped_slope')

In [ ]:
clip_raster_to_geom('/content/drive/MyDrive/disaster_proj/07_resampled/resampled_rain.tif', '/content/drive/MyDrive/disaster_proj/04_buffered_roads/merged_buffered_roads_3000.geojson', '/content/drive/MyDrive/disaster_proj/06_clipped_to_road_buffer/clipped_to_road_buffer_rain.tif' )

Curvature

In [ ]:
extract_and_process_zip('/content/drive/MyDrive/disaster_proj/05_DEM_derivates/curvature_outputs.zip', '/content/drive/MyDrive/disaster_proj/04_buffered_roads/merged_buffered_roads_3000.geojson', '/content/drive/MyDrive/disaster_proj/06_clipped_to_road_buffer/clipped_curvature')

### Everything else is clipped to the road buffer

DEM

In [ ]:
clip_raster_to_geom('/content/drive/MyDrive/disaster_proj/07_resampled/resampled_elevation.tif', '/content/drive/MyDrive/disaster_proj/04_buffered_roads/merged_buffered_roads_3000.geojson', '/content/drive/MyDrive/disaster_proj/06_clipped_to_road_buffer/clipped_to_road_buffer_elevation.tif')

Slope

In [ ]:
clip_raster_to_geom('/content/drive/MyDrive/disaster_proj/07_resampled/resampled_slope.tif', '/content/drive/MyDrive/disaster_proj/04_buffered_roads/merged_buffered_roads_3000.geojson', '/content/drive/MyDrive/disaster_proj/06_clipped_to_road_buffer/clipped_to_road_buffer_slope.tif')

Curvature

In [ ]:
clip_raster_to_geom('/content/drive/MyDrive/disaster_proj/07_resampled/resampled_curvature.tif', '/content/drive/MyDrive/disaster_proj/04_buffered_roads/merged_buffered_roads_3000.geojson', '/content/drive/MyDrive/disaster_proj/06_clipped_to_road_buffer/clipped_to_road_buffer_curvature.tif')

LULC

In [ ]:
clip_raster_to_geom('/content/drive/MyDrive/disaster_proj/07_resampled/resampled_lulc.tif', '/content/drive/MyDrive/disaster_proj/04_buffered_roads/merged_buffered_roads_3000.geojson', '/content/drive/MyDrive/disaster_proj/06_clipped_to_road_buffer/clipped_to_road_buffer_lulc.tif' )

Permafrost

In [ ]:
clip_pf_to_geom('/content/drive/MyDrive/disaster_proj/07_resampled/resampled_permafrost.tif', '/content/drive/MyDrive/disaster_proj/04_buffered_roads/merged_buffered_roads_3000.geojson', '/content/drive/MyDrive/disaster_proj/06_clipped_to_road_buffer/clipped_to_road_buffer_permafrost.tif')

In [ ]:
mask_pf('/content/drive/MyDrive/disaster_proj/06_clipped_to_road_buffer/clipped_to_road_buffer_permafrost.tif', '/content/drive/MyDrive/disaster_proj/06_clipped_to_road_buffer/clipped_to_road_buffer_permafrost_masked.tif')

Precipitation

In [ ]:
clip_raster_to_geom('/content/drive/MyDrive/disaster_proj/07_resampled/resampled_rain.tif', '/content/drive/MyDrive/disaster_proj/04_buffered_roads/merged_buffered_roads_3000.geojson', '/content/drive/MyDrive/disaster_proj/06_clipped_to_road_buffer/clipped_to_road_buffer_rain.tif' )

##9. Classification

In [ ]:
reclassify_raster('/content/drive/MyDrive/disaster_proj/06_clipped_to_road_buffer/clipped_to_road_buffer_permafrost_masked.tif', '/content/drive/MyDrive/disaster_proj/08_reclass_norm/permafrost/permafrost_reclassified.tif')

##10. Normalization

In [ ]:
normalize_lulc_file('/content/drive/MyDrive/disaster_proj/06_clipped_to_road_buffer/clipped_to_road_buffer_lulc.tif', '/content/drive/MyDrive/disaster_proj/08_reclass_norm/lulc')

In [ ]:
input_paths = [
    '/content/drive/MyDrive/disaster_proj/06_clipped_to_road_buffer/clipped_to_road_buffer_elevation.tif',
    '/content/drive/MyDrive/disaster_proj/06_clipped_to_road_buffer/clipped_to_road_buffer_slope.tif',
    '/content/drive/MyDrive/disaster_proj/06_clipped_to_road_buffer/clipped_to_road_buffer_curvature.tif',
    '/content/drive/MyDrive/disaster_proj/06_clipped_to_road_buffer/clipped_to_road_buffer_rain.tif'
]

data_types = ['elevation', 'slope', 'curvature', 'precipitation']

output_folder = '/content/drive/MyDrive/disaster_proj/08_reclass_norm/density'

interpolate(input_paths, output_folder, data_types)

In [ ]:
weighting(
    lulc_file='/content/drive/MyDrive/disaster_proj/08_reclass_norm/lulc/normalized_clipped_to_road_buffer_lulc.tif',
    permafrost_file='/content/drive/MyDrive/disaster_proj/08_reclass_norm/permafrost/permafrost_reclassified.tif',
    density_folder='/content/drive/MyDrive/disaster_proj/08_reclass_norm/density',
    output_zip_file='/content/drive/MyDrive/disaster_proj/weighted_rasters.zip'
)

## 11. Landslide Suscebility

In [ ]:
unzip_file(
    input_zip_file='/content/drive/MyDrive/disaster_proj/weighted_rasters.zip',
    extract_to_folder='/content/drive/MyDrive/disaster_proj/temp_extracted'
)

In [ ]:
fill_permafrost_with_ref_values(
    permafrost_file='/content/drive/MyDrive/disaster_proj/temp_extracted/weighted_permafrost_reclassified.tif',
    reference_file='/content/drive/MyDrive/disaster_proj/temp_extracted/weighted_slope_density.tif',
    output_file='/content/drive/MyDrive/disaster_proj/temp_extracted/filled_permafrost.tif'
)

In [ ]:
raster_files = [
    "/content/drive/MyDrive/disaster_proj/temp_extracted/filled_permafrost.tif",
    "/content/drive/MyDrive/disaster_proj/temp_extracted/weighted_curvature_density.tif",
    "/content/drive/MyDrive/disaster_proj/temp_extracted/weighted_elevation_density.tif",
    "/content/drive/MyDrive/disaster_proj/temp_extracted/weighted_normalized_clipped_to_road_buffer_lulc.tif",
    "/content/drive/MyDrive/disaster_proj/temp_extracted/weighted_precipitation_density.tif",
    "/content/drive/MyDrive/disaster_proj/temp_extracted/weighted_slope_density.tif",
]
output_file = "/content/drive/MyDrive/disaster_proj/09_susceptibility/output.tif"

align_and_sum_rasters(raster_files, output_file)